---
    Gabriel Graells Solé - gabriel.graells01@estudiant.upf.edu
---

# dEFEND - Code Reproduction

In [ ]:
import sys
import csv
csv.field_size_limit(sys.maxsize)
from collections import defaultdict
from random import shuffle
import numpy as np


import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.callbacks import *
from keras.preprocessing.sequence import pad_sequences
from keras.engine.topology import Layer
from keras.backend import epsilon
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import initializers
from keras.metrics import *

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')
PATH = "/content/drive/My Drive/TFG/dEFEND/"

Mounted at /content/drive/


## Data Preprocessing

In [ ]:
def getData() :
    title_file = f'{PATH}data/politifact_title.tsv'
    content_file = f'{PATH}data/politifact_content.tsv'
    comment_file = f'{PATH}data/politifact_comment.tsv'

    news_dic = defaultdict(dict)

    with open(title_file) as tsvfile :
        reader = csv.reader(tsvfile, delimiter=str(u'\t'))
        for line in reader :
            id = line[0]
            news_dic[id]['title'] = line[1]
   
    with open(content_file) as tsvfile :
        reader = csv.reader(tsvfile, delimiter=str(u'\t'))
        next(reader, None)
        for line in reader :
            id = line[0]
            news_dic[id]['label'] = line[1]
            news_dic[id]['content'] = line[2]
            
    with open(comment_file) as tsvfile :
        reader = csv.reader(tsvfile, delimiter=str(u'\t'))
        next(reader, None)
        for line in reader:
            id = line[0]
            news_dic[id]['comments'] = line[1].split('::')

    return news_dic

def train_test_split(_data, test_size = 0.2) :
    id_list = list(_data.keys())
    shuffle(id_list)
    index =  len(id_list) - round(len(id_list) * test_size)
    
    train_x = []
    train_c = []
    train_y = []
    for i in id_list[:index]:
        sentences = []
        sentences.append(_data[i]['title'])
        sentences.extend(_data[i]['content'].split('. '))
        train_x.append(sentences)
        train_c.append(_data[i]['comments'])
        y = _data[i]['label']
        if y:
            train_y.append(np.array([0,1]))
        else:
            train_y.append(np.array([1,0]))
        
    test_x = []
    test_c = []
    test_y = []
    for i in id_list[index:]:       
        sentences = []
        sentences.append(_data[i]['title'])
        sentences.extend(_data[i]['content'].split('.')) 
        test_x.append(sentences)
        test_c.append(_data[i]['comments'])
        y = _data[i]['label']
        if y:
            test_y.append(np.array([0,1]))
        else:
            test_y.append(np.array([1,0]))

    train_y = np.asarray(train_y)
    test_y = np.asarray(test_y)

    return train_x, train_c, train_y, test_x, test_c, test_y

In [ ]:
data = getData()
train_x, train_c, train_y, test_x, test_c, test_y = train_test_split(data, test_size = 0.2)

<center>
    <img src="https://drive.google.com/file/d/17Bn0LRnE44v583ANFkEzChAGv86aPXzt/view?usp=sharing" alt="net" width="400"/>
</center>


As seen in the paper there are two main compenents the **encoder** for words and sentences for comments and news article and, a sentence-comment **co-attention layer**.

The **encoder** is a simple bidirectional GRU unit followed by **attention mechanism** which evaluates the relevance of words or sentences. The **attention mechanism** is implemented as a layer.

The **co-attention layer** will be also implemented as a layer.

## Attention Layer

Layer used for the encoding of words, sentences and comments.

In [ ]:
class AttLayer(Layer) :
    def __init__(self, **kwargs) :
        super(AttLayer, self).__init__(**kwargs)
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = 100
    
    def build(self, input_shape) :
        self.W = tf.Variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = tf.Variable(self.init((self.attention_dim,)))
        self.u = tf.Variable(self.init((self.attention_dim, 1)))
        self._trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    
    def call(self, x, mask=None):
        uit = tf.math.tanh(tf.tensordot(x, self.W, axes = 1) + self.b)
        ait = tf.tensordot(uit, self.u, axes = 1)
        ait = tf.squeeze(ait, -1)
        ait = tf.math.exp(ait)
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= tf.cast(mask, tf.float32)
        
        ait /= tf.cast(tf.math.reduce_sum(ait, axis=1, keepdims=True) + epsilon(), tf.float32)
        ait = tf.expand_dims(ait, axis=-1)
        weighted_input = x * ait
        output = tf.math.reduce_sum(weighted_input, axis=1)

        return output
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

## Co-Attentino Layer

In [ ]:
class Co_attention(Layer):
    '''
        Co-Attention Layer between Senteces and Commnets
    '''
    def __init__(self, **kwargs) :
        self.init = initializers.get('normal')
        self.latent_dim = 200
        self.k = 80
        super(Co_attention, self).__init__(**kwargs)
    
    def build(self, input_shape, mask=None):
        self.Wl = tf.Variable(self.init((self.latent_dim, self.latent_dim)))

        self.Wc = tf.Variable(self.init((self.k, self.latent_dim)))
        self.Ws =  tf.Variable(self.init((self.k, self.latent_dim)))

        self.whs =  tf.Variable(self.init((1, self.k)))
        self.whc =  tf.Variable(self.init((1, self.k)))
        self._trainable_weights = [self.Wl, self.Wc, self.Ws, self.whs, self.whc]

    def call(self, x, mask=None):
        comment_rep = x[0]
        sentence_rep = x[1]
        sentence_rep_trans = tf.transpose(sentence_rep, (0, 2, 1))
        comment_rep_trans = tf.transpose(comment_rep, (0, 2, 1))
        L = tf.math.tanh(tf.einsum('btd,dD,bDn->btn', comment_rep, self.Wl, sentence_rep_trans))
        L_trans = tf.transpose(L, (0, 2, 1))

        Hs = tf.math.tanh(tf.einsum('kd,bdn->bkn', self.Ws, sentence_rep_trans) + tf.einsum('kd,bdt,btn->bkn', self.Wc, comment_rep_trans, L))
        Hc = tf.math.tanh(tf.einsum('kd,bdt->bkt', self.Wc, comment_rep_trans) + tf.einsum('kd,bdn,bnt->bkt', self.Ws, sentence_rep_trans, L_trans))
        As = tf.math.softmax(tf.einsum('yk,bkn->bn', self.whs, Hs))
        Ac = tf.math.softmax(tf.einsum('yk,bkt->bt', self.whc, Hc))
        co_s = tf.einsum('bdn,bn->bd', sentence_rep_trans, As)
        co_c = tf.einsum('bdt,bt->bd', comment_rep_trans, Ac)
        co_sc = tf.concat([co_s, co_c], axis=1)

        return co_sc

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.latent_dim + self.latent_dim)

## Metrics

In [ ]:
class Metrics(Callback):
    def __init__(self,):
        self.log_file = open(f'{PATH}logs/logs.txt', 'a')

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.val_auc = []
        self.val_acc = []

    
    def on_epoch_end(self, epoch, logs={}):
        val_predict_onehot = (
            np.asarray(self.model.predict([self.validation_data[0], self.validation_data[1]]))).round()
        val_targ_onehot = self.validation_data[2]
        val_predict = np.argmax(val_predict_onehot, axis=1)
        val_targ = np.argmax(val_targ_onehot, axis=1)
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        _val_auc = roc_auc_score(val_targ, val_predict)
        _val_acc = accuracy_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        self.val_auc.append(_val_auc)
        self.val_acc.append(_val_acc)
        print("Epoch: %d - val_accuracy: % f - val_precision: % f - val_recall % f val_f1: %f auc: %f" % (
            epoch, _val_acc, _val_precision, _val_recall, _val_f1, _val_auc))
        self.log_file.write(
            "Epoch: %d - val_accuracy: % f - val_precision: % f - val_recall % f val_f1: %f auc: %f\n" % (epoch,
                                                                                                          _val_acc,
                                                                                                          _val_precision,
                                                                                                          _val_recall,
                                                                                                          _val_f1,
                                                                                                          _val_auc))
        return

## Model

In [ ]:
class dEFEND() :
    def __init__(self):
        self.model = None
        self.MAX_SENTENCE_LENGTH = 120
        self.MAX_SENTENCE_COUNT = 50
        self.MAX_COMS_COUNT = 150
        self.MAX_COMS_LENGTH = 120
        self.VOCABULARY_SIZE = 0
        self.metrics = Metrics()
        self.reverse_word_index = None
        self.word_embedding = None 
        self.word_attention_model = None
        self.sentence_comment_co_model = None
        self.tokenizer = None
        self.class_count = 2

        # Variables for calculating attention weights
        self.news_content_word_level_encoder = None
        self.comment_word_level_encoder = None
        self.news_content_sentence_level_encoder = None
        self.comment_sequence_encoder = None
        self.co_attention_model = None

    def init_tokenizer(self, train_x, train_c, test_x, test_c) :
        self.tokenizer = Tokenizer(num_words = 20000)

        all_sentences = []
        all_sentences.extend(train_x)
        all_sentences.extend(test_x)

        all_comments = []
        all_comments.extend(train_c)
        all_comments.extend(test_c)

        all_texts = []
        for sentence in all_sentences:
            all_texts.append(sentence)

        for comments in all_comments:
            for comment in comments:
                all_texts.append(comment)


        self.tokenizer.fit_on_texts(all_texts)
        self.VOCABULARY_SIZE = len(self.tokenizer.word_index) + 1
        self.reverse_word_index = {value: key for key, value in self.tokenizer.word_index.items()}


    def encode_text(self, texts) :
        encoded_texts = np.zeros((len(texts), self.MAX_SENTENCE_COUNT, self.MAX_SENTENCE_LENGTH), dtype='int32')
        for i, text in enumerate(texts):
            encoded_text = np.array(pad_sequences(
                self.tokenizer.texts_to_sequences(text),
                maxlen=self.MAX_SENTENCE_LENGTH, padding='post', truncating='post', value=0))[:self.MAX_SENTENCE_COUNT]
            encoded_texts[i][:len(encoded_text)] = encoded_text
        
        return encoded_texts


    def encode_comments(self, comments) :
        encoded_texts = np.zeros((len(comments), self.MAX_COMS_COUNT, self.MAX_COMS_LENGTH), dtype='int32')
        for i, text in enumerate(comments):
            encoded_text = np.array(pad_sequences(
                self.tokenizer.texts_to_sequences(text),
                maxlen=self.MAX_COMS_LENGTH, padding='post', truncating='post', value=0))[:self.MAX_COMS_COUNT]
            encoded_texts[i][:len(encoded_text)] = encoded_text
        
        return encoded_texts


    def get_embedding(self, embedding_dim):
        embeddings_index = {}
        with open(f'{PATH}data/glove.6B.100d.txt') as file: 
            for line in file:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs

        word_index = self.tokenizer.word_index
        embedding_matrix = np.random.random((len(word_index) + 1, embedding_dim))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        
        return embedding_matrix
        

    def build_model(self, n_classes=2, embedding_dim=100, aff_dim=80):
        embedding_matrix = self.get_embedding(embedding_dim)
        
        embedding_layer = Embedding(len( self.tokenizer.word_index) + 1,
                                    embedding_dim,
                                    weights=[embedding_matrix],
                                    trainable=True,
                                    mask_zero=False)
        
        com_embedding_layer = Embedding(len( self.tokenizer.word_index) + 1,
                                    embedding_dim,
                                    weights=[embedding_matrix],
                                    input_length=self.MAX_SENTENCE_LENGTH,
                                    trainable=True,
                                    mask_zero=False)
        
        # News Content Encoding
        # Word Level
        sentence_input = Input(shape=(self.MAX_SENTENCE_LENGTH,), dtype='int32')
        embedded_sequences = embedding_layer(sentence_input)
        l_lstm = Bidirectional(GRU(100, return_sequences=True), name='word_lstm')(embedded_sequences)
        l_att = AttLayer(name='word_attention')(l_lstm)
        sentEncoder = Model(sentence_input, l_att)
        self.news_content_word_level_encoder = sentEncoder
        # Sentence Level
        content_input = Input(shape=(self.MAX_SENTENCE_COUNT, self.MAX_SENTENCE_LENGTH), dtype='int32')
        content_encoder = TimeDistributed(sentEncoder)(content_input)
        l_lstm_sent = Bidirectional(GRU(100, return_sequences=True), name='sentence_lstm')(content_encoder)
        self.news_content_sentence_level_encoder = Model(content_input, l_lstm_sent)

        # Comments Encoding
        comment_input = Input(shape=(self.MAX_COMS_LENGTH,), dtype='int32')
        com_embedded_sequences = com_embedding_layer(comment_input)
        c_lstm = Bidirectional(GRU(100, return_sequences=True), name='comment_lstm')(com_embedded_sequences)
        c_att = AttLayer(name='comment_word_attention')(c_lstm)
        comEncoder = Model(comment_input, c_att, name='comment_word_level_encoder')
        self.comment_word_level_encoder = comEncoder
        all_comment_input = Input(shape=(self.MAX_COMS_COUNT, self.MAX_COMS_LENGTH), dtype='int32')
        all_comment_encoder = TimeDistributed(comEncoder, name='comment_sequence_encoder')(all_comment_input)
        self.comment_sequence_encoder = Model(all_comment_input, all_comment_encoder)

        # Co-Attention
        L_coattention = Co_attention(name="co-attention")([all_comment_encoder, l_lstm_sent])
        L_Model = Model([all_comment_input, content_input], L_coattention)
        self.co_attention_model = L_Model

        # Output Layer
        preds = Dense(2, activation='softmax')(L_coattention)
        model = Model(inputs=[all_comment_input, content_input], outputs=preds)
        model.summary()

        # Loss 
        optimize = RMSprop(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimize,
                      metrics = [Accuracy()])
        
        return model


    def train(self, train_x, train_y, train_c, test_c, test_x, test_y, batch_size=20, epochs=10, saved_model_file = 'model.tf') :
        self.init_tokenizer(train_x, train_c, test_x, test_c)
        self.model = self.build_model()

        #Encode Input
        encoded_train_x = self.encode_text(train_x)
        encoded_test_x = self.encode_text(test_x)
        encoded_train_c = self.encode_comments(train_c)
        encoded_test_c = self.encode_comments(test_c)
    
        callbacks = []
        callbacks.append(
            ModelCheckpoint(
                filepath= f'{PATH}models/{saved_model_file}',
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=False,
            )
        )
        #callbacks.append(Metrics())
        self.model.fit([encoded_train_c, encoded_train_x], y=train_y,
                       validation_data=([encoded_test_c, encoded_test_x], test_y),
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=1,
                       callbacks=callbacks)
        

    def predict(self, x, c):
        encoded_x = self.encode_text(x)
        encoded_c = self.encode_comments(c)
        return self.model.predict([encoded_c, encoded_x])

    def process_activation_weights(self,  encoded_text, content_word_level_attentions, sentence_co_attention):
        no_pad_text_att = []
        for k in range(len(encoded_text)):
            tmp_no_pad_text_att = []
            cur_text = encoded_text[k]
            for i in range(len(cur_text)):
                sen = cur_text[i]
                no_pad_sen_att = []
                if sum(sen) == 0:
                    continue
                for j in range(len(sen)):
                    wd_idx = sen[j]
                    if wd_idx == 0:
                        continue
                    wd = self.reverse_word_index[wd_idx]
                    no_pad_sen_att.append((wd, content_word_level_attentions[k][i][j]))
                tmp_no_pad_text_att.append((no_pad_sen_att, sentence_co_attention[k][i]))
            no_pad_text_att.append(tmp_no_pad_text_att)

        # Normalize without padding tokens
        no_pad_text_att_normalize = None
        for npta in no_pad_text_att:
            if len(npta) == 0:
                continue
            sen_att, sen_weight = list(zip(*npta))
            new_sen_weight = [float(i) / sum(sen_weight) for i in sen_weight]
            new_sen_att = []
            for sw in sen_att:
                word_list, att_list = list(zip(*sw))
                att_list = [float(i) / sum(att_list) for i in att_list]
                new_wd_att = list(zip(word_list, att_list))
                new_sen_att.append(new_wd_att)
            no_pad_text_att_normalize = list(zip(new_sen_att, new_sen_weight))

        return no_pad_text_att_normalize

    def  process_activation_weights_comments(self, encoded_text, sentence_co_attention):
        no_pad_text_att = []
        for k in range(len(encoded_text)):
            tmp_no_pad_text_att = []
            cur_text = encoded_text[k]
            for i in range(len(cur_text)):
                sen = cur_text[i]
                no_pad_sen_att = []
                if sum(sen) == 0:
                    continue
                for j in range(len(sen)):
                    wd_idx = sen[j]
                    if wd_idx == 0:
                        continue
                    wd = self.reverse_word_index[wd_idx]
                    no_pad_sen_att.append(wd)
                tmp_no_pad_text_att.append((no_pad_sen_att, sentence_co_attention[k][i]))

            no_pad_text_att.append(tmp_no_pad_text_att)

        return no_pad_text_att

    def activation_maps(self, news_article_sentence_list, news_article_comment_list):
        encoded_text = self.encode_text(news_article_sentence_list)
        encoded_comment = self.encode_comments(news_article_comment_list)
        

        # Get weights trainned attention layer for news content
        content_word_level_attentions = []
        W, b, u = self.news_sequence_encoder.layer.get_layer('word_attention').get_weights()
        content_word_encoder = Model(inputs=self.news_sequence_encoder.layer.input, outputs=self.news_sequence_encoder.layer.get_layer('word_lstm').output)
        for sent_text in encoded_text:
            word_level_weights = content_word_encoder.predict(sent_text)

            uit = np.tanh(np.matmul(word_level_weights, W) + b)
            ait = np.matmul(uit, u)
            ait = np.squeeze(ait, -1)
            content_word_wattention = (np.exp(ait) / np.sum(np.exp(ait), axis=1)[:, np.newaxis])
            content_word_level_attentions.append(content_word_wattention)

        # Get the word level attention for comments
        comment_word_level_attentions = []
        W, b, u = self.comment_sequence_encoder.layer.get_layer('comment_word_attention').get_weights()
        comment_word_encoder = Model(inputs=self.comment_sequence_encoder.layer.input,
                                    outputs=self.comment_sequence_encoder.layer.get_layer('comment_lstm').output)
        for comment_text in encoded_comment:
            comment_word_level_weights = comment_word_encoder.predict(comment_text)
            uit = np.tanh(np.matmul(comment_word_level_weights, W) + b)
            ait = np.matmul(uit, u)
            ait = np.squeeze(ait, -1)
            comment_word_level_attention = (np.exp(ait) / np.sum(np.exp(ait), axis=1)[:, np.newaxis])
            comment_word_level_attentions.append(comment_word_level_attention)

        # Get the co attention between document sentences and comments
        comment_level_encoder = Model(inputs=self.comment_sequence_encoder.input,
                                    outputs=self.comment_sequence_encoder.output)
        comment_level_weights = comment_level_encoder.predict(encoded_comment)
        sentence_level_encoder = Model(inputs=self.news_sequence_encoder.input,
                                    outputs=self.news_sequence_encoder.output)

        sentence_level_weights = sentence_level_encoder.predict(encoded_text)
        [Wl, Wc, Ws, whs, whc] = self.co_attention_model.get_weights()

        ### Calculate the co attention
        sentence_rep = sentence_level_weights
        comment_rep = comment_level_weights
        sentence_rep_trans = np.transpose(sentence_rep, axes=(0, 2, 1))
        comment_rep_trans = np.transpose(comment_rep, axes=(0, 2, 1))

        L = np.tanh(np.einsum('btd,dD,bDn->btn', comment_rep, Wl, sentence_rep_trans))
        L_trans = np.transpose(L, axes=(0, 2, 1))

        Hs = np.tanh(np.einsum('kd,bdn->bkn', Ws, sentence_rep_trans) + np.einsum('kd,bdt,btn->bkn', Wc, comment_rep_trans, L))
        Hc = np.tanh(np.einsum('kd,bdt->bkt', Wc, comment_rep_trans) + np.einsum('kd,bdn,bnt->bkt', Ws, sentence_rep_trans, L_trans))
        sent_unnorm_attn = np.einsum('yk,bkn->bn', whs, Hs)
        comment_unnorm_attn = np.einsum('yk,bkt->bt', whc, Hc)
        sentence_co_attention = (np.exp(sent_unnorm_attn) / np.sum(np.exp(sent_unnorm_attn), axis=1)[:, np.newaxis])
        comment_co_attention = (np.exp(comment_unnorm_attn) / np.sum(np.exp(comment_unnorm_attn), axis=1)[:, np.newaxis])
    
        res_comment_weight = self.process_activation_weights_comments(encoded_comment, comment_co_attention)# process_atten_weights not used !!!????
        res_sentence_weight = self.process_activation_weights_comments(encoded_text, sentence_co_attention)

        return res_comment_weight, res_sentence_weight

In [ ]:
dEFEND_Model = dEFEND()
dEFEND_Model.train(train_x, train_y, train_c, test_c, test_x, test_y, batch_size=20, epochs=10, saved_model_file = 'model.tf')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 50, 120)]    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 150, 120)]   0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 50, 200)      9693300     input_2[0][0]                    
__________________________________________________________________________________________________
comment_sequence_encoder (TimeD (None, 150, 200)     9693300     input_4[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/My Drive/TFG/dEFEND/models/model.tf/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/TFG/dEFEND/models/model.tf/assets


Epoch 2/10
17/17 [==============================] - 7s 434ms/step - loss: 1.7352e-04 - accuracy: 0.0000e+00 - val_loss: 9.2850e-05 - val_accuracy: 0.0000e+00


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/TFG/dEFEND/models/model.tf/assets


Epoch 10/10
17/17 [==============================] - 7s 438ms/step - loss: 2.5826e-07 - accuracy: 0.0000e+00 - val_loss: 1.2065e-07 - val_accuracy: 0.0000e+00


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/My Drive/TFG/dEFEND/models/model.tf/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/TFG/dEFEND/models/model.tf/assets
